In [1]:
# !pip install web3
# !pip install python-dotenv
# !pip install requests
# !pip install pandas

## Imports

In [2]:
from web3 import Web3

# For enums
from enum import StrEnum, auto

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

import requests
import json

# Importing Pandas to create DataFrame
import pandas as pd

# For timestamp conversion
import time

## Constants

In [3]:
# Contract types
class Contract_Type(StrEnum):
    SMART = auto()
    NFT = auto()

# Events
class Event(StrEnum):
    BURN = 'Burn'
    COLLECT = 'Collect'
    MINT = 'Mint'
    INCREASE_LIQUIDITY = 'IncreaseLiquidity'
    DECREASE_LIQUIDITY = 'DecreaseLiquidity'
    
# Interested events for the smart contract
CONTRACT_EVENTS = {
    Contract_Type.SMART : [Event.BURN, Event.COLLECT, Event.MINT],
    Contract_Type.NFT: [Event.COLLECT, Event.INCREASE_LIQUIDITY, Event.DECREASE_LIQUIDITY]
}

# Etherscan endpoint
ETHERSCAN_ENDPOINT = 'https://api.etherscan.io/api'

# Token 0 name, decimals and display (decimals)
TOKEN0_NAME = 'USDC'
TOKEN0_DECIMALS = 6
TOKEN0_DISPLAY = 2

# Token 1 name and decimals
TOKEN1_NAME = 'WETH'
TOKEN1_DECIMALS = 18

# Transaction hash we are interested
TRAN_HASH = '0x8bab544f6f87449d25b9e4cef8e2d59f1771050b10cae2c17af1f35eceeffdf9'

## Load environment variables

In [4]:
dotenv_path = Path('.env/logs')
load_dotenv(dotenv_path=dotenv_path)
PROVIDER_URL = os.getenv('PROVIDER_URL')
SMART_CONTRACT_ADDR = os.getenv('SMART_CONTRACT')
NFT_CONTRACT_ADDR = os.getenv('NFT_CONTRACT')
ETHERSCAN_API_KEY = os.getenv('ETHERSCAN_API_KEY')
SMART_CONTRACT_ABI = os.getenv('SMART_CONTRACT_ABI')

## Load the ABIs

In [5]:
# Load the previously downloaded smart contract; unable to download it - Contract source code not verified
with open(SMART_CONTRACT_ABI) as f:
    smart_abi = json.load(f)

# Get ABI of contract; use etherscan API for verifiable smart contracts ABI
params = {'module':'contract', 'action':'getabi', 'address':{NFT_CONTRACT_ADDR}, 'apikey':{ETHERSCAN_API_KEY}}
response = json.loads(requests.get(url=ETHERSCAN_ENDPOINT, params=params).text)
if response['status'] == '1':
    nft_abi = json.loads(response['result'])
else:
    nft_abi = None

## Create Smart and NFT contracts

In [6]:
# Initialize web3 instance
web3 = Web3(Web3.HTTPProvider(PROVIDER_URL))
smart_contract = web3.eth.contract(address=web3.to_checksum_address(SMART_CONTRACT_ADDR), abi=smart_abi)
nft_contract = web3.eth.contract(address=web3.to_checksum_address(NFT_CONTRACT_ADDR), abi=nft_abi)

## Calculate Event Signatures

In [7]:
def event_signature(abi:str, type:Contract_Type) -> {str:str}:
    """ Return event singatures for given ABI type
    Parameters:
    abi : str
        ABI contract
    type : Contract_Type
        Either SMART or NFT
        
    Returns:
    dict
        A dictionary signature hex -> event name
    """
    # Dictionary to return (hex -> name)
    signature_to_name = {}
    
    # Loop through abi skipping the first one
    for item in abi[1:]:
        ## Only interested in Event types and events in scope
        if ((item['type'] == 'event') and (item['name'] in CONTRACT_EVENTS[type])):
            name = item['name']
            inputs = [param['type'] for param in item['inputs']]
            inputs = ','.join(inputs)
            # Hash event signature
            event_signature_text = f'{name}({inputs})'
            event_signature_hex = web3.to_hex(web3.keccak(text=event_signature_text))
            signature_to_name[event_signature_hex] = name
    return signature_to_name

## Event Signatures for Smart and NFT Contracts

In [8]:
# Dict to store signatures signature -> event name
event_signatures = {}
event_signatures[Contract_Type.SMART] = event_signature(smart_abi, Contract_Type.SMART)
event_signatures[Contract_Type.NFT] = event_signature(nft_abi, Contract_Type.NFT)
event_signatures

{<Contract_Type.SMART: 'smart'>: {'0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c': 'Burn',
  '0x70935338e69775456a85ddef226c395fb668b63fa0115f5f20610b388e6ca9c0': 'Collect',
  '0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde': 'Mint'},
 <Contract_Type.NFT: 'nft'>: {'0x40d0efd1a53d60ecbf40971b9daf7dc90178c3aadc7aab1765632738fa8b8f01': 'Collect',
  '0x26f6a048ee9138f2c0ce266f322cb99228e8d619ae2bff30c67f8dcf9d2377b4': 'DecreaseLiquidity',
  '0x3067048beee31b25b2f1681f88dac838c8bba36af25bfb2b7cf7473a5847e35f': 'IncreaseLiquidity'}}

## Utility method to create a decoded log

In [9]:
def create_decoded_log(key:str, search_type:Contract_Type, log:dict) -> dict:
    """ Return a decode log constructed from given raw log event. Each decoded log is a dictionary of values
    Parameters:
    key : str
        key to the event signature
    search_type : Contract_Type
        Either SMART or NFT

    log : dict
        Unprocessed log event
        
    Returns:
    dict
        A decoded log event
    """
    # The decoded log to return
    decoded_log = None

    # Set the contract based on search type
    contract = smart_contract if (search_type == Contract_Type.SMART) else nft_contract
    
    # Get the corresponding event name for the signature key
    event_name = event_signatures[search_type][key]
    
    # Process the using event name signature
    processed_log = contract.events[event_name]().process_log(log)
    amount0_fees = processed_log['args']['amount0']
    amount1_fees = processed_log['args']['amount1']
    log_index = processed_log['logIndex']
    if search_type == Contract_Type.NFT:
        token_id = processed_log['args']['tokenId']
        decoded_log = {'token_id':token_id, 'log_index':log_index, 'amount0':amount0_fees,
                       'amount1':amount1_fees, 'event':event_name}
    else:
        decoded_log = {'log_index':log_index, 'amount0':amount0_fees, 'amount1':amount1_fees,
                       'event':event_name}
    return decoded_log

## Process Event Logs

In [10]:
def process_logs(tran_hash:str, search_type:Contract_Type) -> list[dict]:
    """ Return list of processed logs. Each log is a dictionary of values
    Parameters:
    tran_hash : str
        transaction hash
    search_type : Contract_Type
        Either SMART or NFT
        
    Returns:
    list
        A list of decoded logs
    """
    
    # List of decoded logs
    decoded_logs = []
    
    # Get transaction receipt
    receipt = web3.eth.get_transaction_receipt(tran_hash)

    # Get the transaction from the block
    time_stamp = web3.eth.get_block(receipt['blockNumber'])['timestamp']
    
    for log in receipt.logs:
        # Is it a smart contract search?
        if search_type == Contract_Type.SMART:                
            # Only interested in our smart contract addresses
            if log.address != SMART_CONTRACT_ADDR: continue
            # Key to search in the signature dict
            key = web3.to_hex(log['topics'][0])
            # Filter out other smart events not found in the signature map
            if key not in event_signatures[search_type]: continue
        else:
            # Only interested in our NFT contract addresses
            if log.address != NFT_CONTRACT_ADDR: continue
            # Key to search in the signature dict
            key = web3.to_hex(log['topics'][0])
            # Filter out other NFT events not found in the signature map
            if key not in event_signatures[search_type]: continue
        # At this point, we have either smart or nft event we are interested
        decoded_log = create_decoded_log(key=key, search_type=search_type, log=log)
        decoded_log['time_stamp'] = int(time_stamp)
        decoded_logs.append(decoded_log)
    return decoded_logs

## Create a DataFrame

In [11]:
def create_DF(search_type:Contract_Type) -> pd.DataFrame:
    """ Return a DF created for given search type
    Parameters:
    search_type : Contract_Type
        Either SMART or NFT
        
    Returns:
    pd.DataFrame
        A Dataframe
    """
    decoded_logs = process_logs(TRAN_HASH, search_type)
    # Create a DF from list of transactions
    df = pd.DataFrame.from_records(data=decoded_logs)
    if search_type == Contract_Type.SMART:
        df.columns = ['LogIndex', TOKEN0_NAME, TOKEN1_NAME, 'Type', 'Timestamp']
    else:
        df.columns = ['TokenId', 'LogIndex', TOKEN0_NAME, TOKEN1_NAME, 'Type', 'Timestamp']

    df['Timestamp'] = df['Timestamp'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(x)))
    # Apply token decimals
    token_divisor = int('1' + ('0' * TOKEN0_DECIMALS))
    df[TOKEN0_NAME] = df[TOKEN0_NAME].apply(lambda x : x / token_divisor).round(2)
    token_divisor = int('1' + ('0' * TOKEN1_DECIMALS))
    df[TOKEN1_NAME] = df[TOKEN1_NAME].apply(lambda x : x / token_divisor)
    return df

In [12]:
df = create_DF(search_type=Contract_Type.NFT)
# Apply styles
df_styler = df.style.set_properties(subset=['Type'],**{'text-align': 'left'})
df_styler = df_styler.format('{:.2f}', subset=[TOKEN0_NAME])
df_styler.set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

,TokenId,LogIndex,USDC,WETH,Type,Timestamp
0,927456,43,0.00,2.983947,DecreaseLiquidity,2025-02-26 19:06:35
1,927456,47,94.18,3.021172,Collect,2025-02-26 19:06:35
2,937002,71,3308.10,1.497818,IncreaseLiquidity,2025-02-26 19:06:35
